# Use AI classifier to spot events

We will see how a trained AI on simulation data is able to spot a dyfunction, here namely a broken fan, in operation data.

### Load the AI model

In [42]:
import numpy as np
import pandas as pd
import pickle

# load the already trained classifier and
classifier = pickle.load(open("data/broken_fan_classifier.pkl", "rb"))
classifier_mean = pd.read_csv("data/broken_fan_classifier_mean.csv", index_col=0)
classifier_std = pd.read_csv("data/broken_fan_classifier_std.csv", index_col=0)

### Load the operation data

In [43]:
# load the operation data to check for borken fan event
data = pd.read_csv("data/cpu_hot_day_intensive_use_broken.csv", index_col=0)
# do data normalization
classifier_data = (
    data[["fan.T_air", "T_cpu", "fan.tension"]] - classifier_mean.values.T
) / classifier_std.values.T
classifier_data.T

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
fan.T_air,-0.126391,-0.135368,-0.121939,-0.143242,-0.107510,-0.156307,-0.158709,-0.168327,-0.134351,-0.136329,...,-0.121656,-0.158946,-0.116243,-0.090121,-0.122457,-0.141683,-0.113959,-0.150109,-0.145164,-0.123614
T_cpu,-0.605211,0.076345,0.724226,1.277194,1.891320,2.449718,2.955452,3.481764,3.962959,4.451453,...,9.126444,8.920820,8.799973,8.603947,8.442281,8.293733,8.162098,7.987317,7.881552,7.702409
fan.tension,-0.661952,-0.661952,-0.661952,-0.661952,-0.661952,-0.661952,0.786521,0.786521,0.786521,0.786521,...,2.234994,2.234994,2.234994,2.234994,2.234994,2.234994,2.234994,2.234994,2.234994,2.234994


### Perform the checks

In [44]:
# use the AI classifier to predict nominal vs broken fan
diag = pd.DataFrame(
    np.stack((classifier_data.index, classifier.predict(classifier_data.values)), axis=1),
    columns=["time", "fan_working_diagnostic"],
)

# also get associated statistics
classifier_proba = pd.DataFrame(
    np.hstack(
        (np.arange(0, 31, 1).reshape(-1, 1), classifier.predict_proba(classifier_data.values))
    ),
    columns=["time", "non_working_proba", "working_proba"],
)
non_working_proba = classifier_proba[["time", "non_working_proba"]].rename(
    columns={"non_working_proba": "proba"}
)
working_proba = classifier_proba[["time", "working_proba"]].rename(
    columns={"working_proba": "proba"}
)

In [45]:
from cpu.utils.plot_recorders import plot_recorders

plot_recorders(
    {
        "diag": diag,
        "working_diag_proba": working_proba,
        "non_working_diag_proba": non_working_proba,
    },
    [[("time", "fan_working_diagnostic"), ("time", "proba")]],
    width=800,
    height=600,
    legend={"x": 0.75, "y": 0.25, "font": {"size": 20}, "orientation": "h", "xanchor": "center"},
)

    'data': [{'mode': 'markers',
              'name': 'diag',
  …

In [46]:
diag.to_csv(f"data/fan_working_diagnostic.csv", index=False)